In [ ]:
import numpy as np
from sympy  import Symbol,Add,Mul,Pow,Number,symbols
from sympy import  factor,srepr,sympify,expand,sqrt,simplify
import matplotlib.pyplot as plt 

In [ ]:
def max_sum(clause):
    max=0
    if clause.func == Add :
        for t in clause.args :
            if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) 
                #print(max)
            
            if t.func == Mul :
                if  isinstance(t.args[0], Symbol)  :           ###changed  from  previous one
                    max= max+1

                if isinstance(t.args[0], Number) and t.args[0] > 0:
                    max = max + (int(t.args[0]))
                    
                if isinstance(t.args[0], Number) and t.args[0] < 0:
                    max = max + 0
                      
                     
                    
                if isinstance(t.args[0], Number) and  t.args[0] <0 :
                    
                    pass  
            if t.func == Symbol  :
                
                max=max+1
                
            #if isinstance(t ,Symbol):
             #   max=max+1
            
            '''if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) '''
              
                

            #else :
             #   max = max+1
      

    elif clause.func == Mul :
        if  isinstance(clause.args[0], Number) and  clause.args[0] >0 :
            max =max+ int(clause.args[0])
        if   isinstance(clause.args[0], Number) and  clause.args[0] <0 :
            max =max+ int(0)
            
        else :
            max=max+1

    elif clause.func == Symbol :
        max = 1
    elif isinstance(clause,Number):
        max=max+int(clause)
    

    return max

In [ ]:
def rule_2(clause, expression):
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]
    return expression

def  rule_3(clause , expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') * Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]]=1
            expression[list(clause.free_symbols)[0]]=1
    return expression


def  rule_4(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1

    return expression  
def rule_5(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [ ]:
def create_dictionary_robust(m):
    m_bin=bin(m)[2:][::-1]
    m_dict={}
    for i,j in enumerate(m_bin):
        m_dict[i] =int (j)

    p_dict={}
    p_dict[0]=1
    
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict1={}
    z_dict2={}

    for i in range(2,len(m_bin)):
        for j in range(i-int(np.floor(np.log2(i))),i):
            if i!= j:
                z_dict1[(j, i)] =  Symbol('z'+str(j)+str(i))

    for i in range(1,len(m_bin)-1):
        for j in range(i,i+len(bin(i)[2:][::-1])+1):
            if i!= j  and j <= len(m_bin):
                z_dict2[(i, j)] =  Symbol('z'+str(i)+str(j))




    return m_dict , p_dict ,q_dict , z_dict2
                



    


In [ ]:
def create_clause1(m,p,q,z):
    clauses=[]
    clauses1=[]
    z_zero={}
    n=len(m)+int(np.ceil(len(m)/2)) -1
    for i in range(n):
        clause=0
        for j in range(i+1):
            clause+=q.get(j,0) * p.get(i-j,0)
        #clause  += - m.get(i,0)
        for j in range(i+1):
            clause += z.get((j, i), 0)
        
        if type(clause) == int:
            clause = sympify(clause)
        if clause != 0:
            max_sum1=max_sum(clause)
            if max_sum1 !=0  :
                max_carry = int(np.floor(np.log2(max_sum1)))
            else : 
                max_carry  = 0
        for j in range(len(z)):
            if j-i > max_carry:
                if z.get((i-1, j), 0) != 0:
                    z[(i-1, j)] = 0
                    z_zero['z'+str(i-1)+str(j)] = 0 
        clause  += - m.get(i,0)
        for j in range(i+1, i+5):
            if j-i <=  max_carry +1 :
               clause += - 2**(j-i) * z.get((i, j), 0) 
        
        
        if clause ==0 :
            clause = sympify(clause)
        clauses.append(clause)

    #for clause in clauses:
     #   for keys in z_zero:
     #       clause=str(clause).replace(z_zero[keys],'0')
     #   clauses1.append(clause)
            

    return clauses

In [ ]:
def rule_21(clause, expression):
    
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            expression[list(clause.free_symbols)[0] + list(clause.free_symbols)[1]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[lisit(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule21applied'))
    print(expression)
    return expression

In [ ]:
def rule_31(clause, expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==1 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')  -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule31applied'))
    print(expression)
    return expression

In [ ]:
def rule_xpow2(clause):

    '''x*x = x'''
    '''for i in clause.args:
        for p in range(1,len(i.args)):
            if i.args[p] == i.args[p-1]:
                expression[i.args[p-1]* i.args[p]]= i.args[p]'''

    clause = sympify(str(clause) . replace('**2',''))
    return clause
             


In [ ]:
def rule_311(clause,expression):
    if isinstance(clause,Symbol):
        expression[clause] = 0
        print(expression)
    return expression

In [ ]:
def rule_322(dict1,dict2,expression):
    '''p3+q3=1  and p3 is in dictinary . so itn will get automatucally 1'''
    if len(dict2) !=0 :
        if 'p' in str(list(dict2.keys())[0]):
            if dict1[Symbol(str(list(dict2.keys())[0])) + Symbol('q'+str(list(dict2.keys())[0])[-1])]==1:
                expression[Symbol('q'+str(list(dict2.keys())[0])[-1])]= 1-dict2[Symbol(str(list(dict2.keys())[0]))]
                                         
    #print(dict1[Symbol(str(list(dict2.keys())[0])) + Symbol('q'+str(list(dict2.keys())[0])[-1])])
    return expression

     
    
        


    
    
    

In [ ]:
def rule_11(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) >0 :
        for  t in negative:
            if   -t.args[0] > max_sum(clause):
                var= t/t.args[0]
                expression[var] =0
    if (len(expression)!= 0):
        print(('rule11applied'))
    print(expression)
    #print (max_sum(clause))

    return expression

In [ ]:
def rule_51(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [ ]:
def  rule_41(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1
    return expression

In [ ]:
def  rule_61(clause,expression):
    if clause.func == Symbol   and  len(list(clause.free_symbols)) == 1:
        '''if isinstance(clause.args[0],Number):
            var= clause / clause.args[0]
            sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
            rule= Symbol('x')
            if sub_clause -rule ==0 :
               expression[list(clause.free_symbols)[0]] =0'''

        print(clause.free_symbols)
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
           expression[list(clause.free_symbols)[0]] =0

    if clause.func == Mul  and   len(list(clause.free_symbols)) == 1: 
        var= clause / clause.args[0]
        sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =0
    return expression


In [ ]:
def rule_71(clause,expression):
    count=0
    constant =0
    negative=[]
    if clause.func==Add  :
        for t in clause.args:
            if isinstance(t, Number) and int(t) > 0:
                constant= constant + int(t)
                print(type(int(t)))
            if  isinstance(t,Symbol):
                count = count +1
                print(type(count))
            if t.func==Mul : 
                if isinstance(t.args[0],Number) and  t.args[0] < 0:

                    negative.append(- t.args[0])
                else :

                    count =count +1
                
        print(negative)


    if len(negative) > 0 and constant + count == max(negative  ) and  int(count)  > 0:
        for t in clause.args:
            if t.func == Mul  and isinstance(t.args[0] , Number) :
                var=t/t.args[0]
                expression[var] = 1
                for i in var.args:
                    expression[i] =1
            if t.func == Mul and isinstance(t.args[0], Symbol):
                for i in t.args :
                    if isinstance(i, Symbol):
                       expression[i]=1

    
            
    return expression

In [ ]:
def inverse_dictionray(value,p_dict):
    for i in p_dict.keys():
        if p_dict[i] == value :
            return i




In [ ]:
def rule_81(clause,expression):
    '''2*p4 + z79 + z89 - 2*z910 - 1'''
    negative_constant=[]
    coefficient={}
    negative_coefficient={}
    positive_constant=[]

     
    if clause.func == Add :
        for i in clause.args:
            if isinstance(i,Number) and int(i) < 0 :
                negative_constant.append(-int(i))
        for i in clause.args:
            if isinstance(i,Number) and int(i) > 0 :
                positive_constant.append(int(i))


        for i in clause.args:
            if i.func==Mul and isinstance(i.args[0],Number) and int(i.args[0])>0 :
                var=i/int(i.args[0]) 
                coefficient[var] = int(i.args[0])

            if i.func==Mul and isinstance(i.args[0],Number) and int(i.args[0])<0 :
                var=i/int(i.args[0]) 
                negative_coefficient[var] = -int(i.args[0])

            if i.func== Mul and isinstance(i.args[0],Symbol):
                coefficient[i] = 1
            if i.func!= Mul and isinstance(i,Symbol):
                coefficient[i] =1 
    p=list(coefficient.values())
    q=list(negative_coefficient.values())
    
    if len(negative_constant)>0 :
        for i in range(len(p)):
           

            if sum(p)- p[i] < max(negative_constant):
               expression[inverse_dictionray(p[i],coefficient)]=1
            if sum(p) == max(negative_constant):
                '''3*p3 -3    p3 should be 1'''
                expression[inverse_dictionray(p[i],coefficient)] =1
    if len(negative_constant) == 0  and len(positive_constant)==0:
        for i in range(len(p)):
            if sum(p)   == min(negative_coefficient):
                '''p1 + q1 - 2*z12'''
                expression[inverse_dictionary(p[i],negative_coefficient)]= inverse_dictionary(p[0],negative_coefficient)
                '''if p1 = q1 then check p2+q2 =1 and backtrack . first p[i]'+q[i] =1 you can assume p[i]=0,q[i]=1 safely'''
                 
            

    if len(positive_constant) > 0 :
        for i in range(len(p)):
            '''p2 - 2*z34 + 1'''
            if p[i] + sum(positive_constant) == max(q):
                expression[inverse_dictionray(p[i],coefficient)] =1
                expression[inverse_dictionray(q,negative_coefficient)] =1
                



    if len(negative_constant)>0 :
        '''2*p3 + p4 + z68 + z78 - 2*z89 - 4  where  negative is 6 but it doesnot have 6 term so z89 will be 0'''
        for i in range(len(q)):
           

           if sum(p) < max(negative_constant) +q[i]:
               expression[inverse_dictionray(q[i],negative_coefficient)]=0
    return expression



    


In [ ]:
def check_for_first_applying_symmetryrule(expression,m):
    '''constraint of the function  it check one p at a time.
    dictionary should 1 p generalize 'later so that it can be  done for arbirary number of p in dictionary '''
    expression1={}
    for j in range(1,m):
        for i in len(expression):
            if expression[Symbol('p'+str(j))]== Symbol('q'+str(j)):
                expression1[Symbol('p'+str(j+1))+Symbol('q'+str(j+1))]=1

    

In [ ]:
expression_prime={}
rule_81(renew_clause33[8],expression_prime)
expression_prime


In [ ]:
m,p,q,z = create_dictionary_robust(3599)
p2=create_clause1(m,p,q,z)
p2
p



In [ ]:
expression_3599={}
for clauses in p2:
    
    rule_11(clauses,expression_3599)


In [ ]:
renew_clause11=[]
for clauses in p2:
    renew_clause11.append(clauses.subs(expression_3599).expand())
renew_clause11

In [ ]:

for clauses in renew_clause11:
    rule_11(clauses,expression_3599)

In [ ]:
renew_clause12=[]
for clauses in renew_clause11:
    renew_clause12.append(clauses.subs(expression_3599).expand())
renew_clause12

In [ ]:
expression_3599[Symbol('p1')] =0
expression_3599[Symbol('q1')]  =1 

In [ ]:
renew_clauses13=[]
for clauses in renew_clause12 :
    rule_21(clauses,expression_3599)

for clauses in renew_clause12 :
    renew_clauses13.append(clauses.subs(expression_3599).expand())
renew_clauses13

In [ ]:
renew_clauses14=[]
for clauses in renew_clauses13 :
    rule_11(clauses,expression_3599)

for clauses in renew_clauses13 :
    renew_clauses14.append(clauses.subs(expression_3599).expand())
renew_clauses14


In [ ]:
renew_clauses15=[]
for clauses in renew_clauses14 :
    rule_41(clauses,expression_3599)

for clauses in renew_clauses14 :
    renew_clauses15.append(clauses.subs(expression_3599).expand())
renew_clauses15

In [ ]:
renew_clauses16=[]
'''for clauses in renew_clauses15 :
    rule_41(clauses,expression_3599)'''

for clauses in renew_clauses15 :
    renew_clauses16.append(clauses.subs(expression_3599).expand())
renew_clauses16

In [ ]:
renew_clause17=[]
expression_4002 = {}
expression_4002[Symbol('p3')  + Symbol('q3')] = 1
expression_4002[Symbol('p3') * Symbol('q3')] = 0
for clauses in renew_clauses16 :
    renew_clause17.append(clauses.subs(expression_4002))
renew_clause17    


In [ ]:
renew_clause18=[]
expression_4003 = {}
for clause in renew_clause17:
    rule_21(clause,expression_4003)
for clause in renew_clause17:
    renew_clause18.append(clause.subs(expression_4003))
renew_clause18   


In [ ]:
expression_4004={}
expression_4005={}
renew_clause19 = []
renew_clause20=[]
for i in range(1,5):
    if i!=3 :
       expression_4004[Symbol('p'+str(3))*Symbol('q'+str(i)) + Symbol('p'+str(i))*Symbol('q'+str(3)) ] = 1  # it is only in maxsum rule because if p1 +q1 = 1 then p1q2 + q1p2 = 1  because one of p1 and q1 is 0 . but it can be 0  but it can't be 2 so it's maximum sum 1.it;s only for rule_11
for clause in renew_clause18:
    renew_clause19.append(clause.subs(expression_4004))
for clause in renew_clause19:
    rule_11(clause,expression_4005)
for clause in renew_clause18:
    renew_clause20.append(clause.subs(expression_4005))
renew_clause20        
    

In [ ]:
expression_4006={}
for clause in renew_clause20:
    rule_311(clause,expression_4006)
str(list(expression_4006.keys())[0])

In [ ]:
expression_4007={}
rule_322(expression_4003,expression_4006,expression_4007)

#Symbol(str(list(expression_4006.keys())[0])) + Symbol('q'+str(list(expression_4006.keys())[0])[-1])

In [ ]:
renew_clause21=[]
renew_clause22=[]



for clause in renew_clause20:
    renew_clause21.append(clause.subs(expression_4006))
renew_clause21 
for clause in renew_clause21:
    renew_clause22.append(clause.subs(expression_4007))
renew_clause22     


In [ ]:
expression_4008={}
expression_4009={}
renew_clause23 = []
renew_clause24=[]
for i in range(1,5):
    if i!=3 :
       expression_4004[Symbol('p'+str(3))*Symbol('q'+str(i)) + Symbol('p'+str(i))*Symbol('q'+str(3)) ] = 1  # it is only in maxsum rule because if p1 +q1 = 1 then p1q2 + q1p2 = 1  because one of p1 and q1 is 0 . but it can be 0  but it can't be 2 so it's maximum sum 1.it;s only for rule_11
for clause in renew_clause22:
    renew_clause23.append(clause.subs(expression_4004))
for clause in renew_clause23:
    rule_11(clause,expression_4008)
for clause in renew_clause22:
    renew_clause24.append(clause.subs(expression_4008))
renew_clause24        

In [ ]:

expression_4009={}
renew_clause25 = []
renew_clause26=[]
for i in range(1,5):
    if i!=3 :
       expression_4004[Symbol('p'+str(3))*Symbol('q'+str(i)) + Symbol('p'+str(i))*Symbol('q'+str(3)) ] = 1  # it is only in maxsum rule because if p1 +q1 = 1 then p1q2 + q1p2 = 1  because one of p1 and q1 is 0 . but it can be 0  but it can't be 2 so it's maximum sum 1.it;s only for rule_11
for clause in renew_clause24:
    renew_clause25.append(clause.subs(expression_4004))
for clause in renew_clause25:
    rule_11(clause,expression_4009)
for clause in renew_clause24:
    renew_clause26.append(clause.subs(expression_4009))
renew_clause26        

In [ ]:
expression_4010={}
renew_clause27 = []
renew_clause28=[]
for i in range(1,5):
    if i!=3 :
       expression_4004[Symbol('p'+str(3))*Symbol('q'+str(i)) + Symbol('p'+str(i))*Symbol('q'+str(3)) ] = 1  # it is only in maxsum rule because if p1 +q1 = 1 then p1q2 + q1p2 = 1  because one of p1 and q1 is 0 . but it can be 0  but it can't be 2 so it's maximum sum 1.it;s only for rule_11
for clause in renew_clause26:
    renew_clause27.append(clause.subs(expression_4004))
for clause in renew_clause27:
    rule_11(clause,expression_4010)
for clause in renew_clause26:
    renew_clause28.append(clause.subs(expression_4010))
renew_clause28        

In [ ]:
for clauses in renew_clause28:
    if invalid_clause(clauses) == False :
        print('false')
    else:
        print('true')

In [ ]:
def invalid_clause(clause):
    '''if we keep  clause variables in the dictionary then it's fine.but if we keep the co efficient then 
    then it is great. because if the clause is  invalid then  it will be no matter what variable,so coefficient and their sum will ensure it'''
    positive_constant=[]
    negative_constant= []
    negative=[]
    positive=[]
    if clause.func == Add  :
        
        for i in clause.args:
            if isinstance(i,Number) and int(i) > 0  :
                positive_constant.append(int(i))

        
            if i.func == Mul and isinstance(i.args[0] , Number) and i.args[0] < 0 :
                negative.append(int(i.args[0]))
            if isinstance(i,Number)  and int(i) < 0:
                negative_constant.append(int(i))
            if   (i.func==Mul and isinstance(i.args[0],Symbol)) :
                positive.append(1)
            if (i.func == Mul and isinstance(i.args[0],Number)) and int(i.args[0])>0:
                positive.append(int(i.args[0]))

            if isinstance(i,Symbol):  #problems: if term  
                positive.append(1)
    if clause.func != Add and   clause.func != Mul and int(clause) != 0 :
        print('yes')
        negative_constant.append(int(clause))


          
    
    
    p_c=sum(positive_constant)
    n_c=sum(negative_constant)
    n=sum(negative)
    p=sum(positive)
    p_cl=len(positive_constant)
    n_cl=len(negative_constant)
    n_l=len(negative)
    p_l=len(positive)
            
    if p_l > 0 and p_cl>0 and n_l == 0 and n_cl ==0 :
            return False
    if  p_l == 0 and p_cl>0 and n_l ==0 and n_cl ==0: 
            return False
    if p_l == 0 and p_cl==0 and n_l == 0 and n_cl > 0:
            return False
    if len(clause.args)== 1 and isinstance(clause.args,Number) and int(clause.args) != 0 :
            return False
    if  n_l == 0 and p_cl==0  and  -n_c > p:
        return False 
    
    else :
            return True




In [ ]:
renew_clause29=[]
expression_4011 = {}
expression_4011[Symbol('p4')  + Symbol('q4')] = 1
expression_4011[Symbol('p4') * Symbol('q4')] = 0
for clauses in renew_clauses16 :
    renew_clause29.append(clauses.subs(expression_4011))
renew_clause29  


In [ ]:
expression_4012={}
renew_clause30=[]
for clauses in renew_clause29:
    rule_41(clauses,expression_4012)
for clauses in renew_clause29:
    renew_clause30.append(clauses.subs(expression_4012))
renew_clause30

    

In [ ]:
expression_4013={}
renew_clause31=[]
for clauses in renew_clause30:
    rule_41(clauses,expression_4013)
for clauses in renew_clause30:
    renew_clause31.append(clauses.subs(expression_4013))
renew_clause31

    

In [ ]:
for clauses in renew_clause31:
    if invalid_clause(clauses) == False :
        print('false')
    else:
        print('true')

In [ ]:
record={}
record['p3 + q3 -1'] = 'false'
record['p3q3'] = 'false'
record['p4 + q4 -1'] = 'false'
record['p4q4'] = 'false'
record

In [ ]:
expression_4014={}
renew_clauses31=[]
expression_4014[Symbol('q3')] =  Symbol('p3')
expression_4014[Symbol('q4')] = Symbol('p4')
expression_4014
for clauses in renew_clauses16:
    renew_clauses31.append(clauses.subs(expression_4014))

renew_clauses31    

In [ ]:
renew_clause32 =[]
for clause in renew_clauses31:
    renew_clause32.append(rule_xpow2(clause))
    
renew_clause32

In [ ]:
expression_4015={}
renew_clause33=[]
for clauses in renew_clause32:
    rule_41(clauses,expression_4015)
for clauses in renew_clause32:
    renew_clause33.append(clauses.subs(expression_4015))
renew_clause33

In [ ]:
expression_4020={}
renew_clause40=[]
for clauses in renew_clause33:
    rule_81(clauses,expression_4020)
for clauses in renew_clause33:
    renew_clause40.append(clauses.subs(expression_4020))
print(expression_4020)
renew_clause40

In [ ]:
expression_4021={}
renew_clause41=[]
for clauses in renew_clause40:
    rule_81(clauses,expression_4020)
for clauses in renew_clause40:
    renew_clause41.append(clauses.subs(expression_4020))
print(expression_4021)
renew_clause41

In [ ]:
expression_4022={}
renew_clause42=[]
for clauses in renew_clause41:
    rule_81(clauses,expression_4020)
for clauses in renew_clause41:
    renew_clause42.append(clauses.subs(expression_4020))
print(expression_4022)
renew_clause42

In [ ]:
def rulecheck_1(clauses,expression):
    constant = 0
    negative=[]
    if clauses.func == Add :
        
        for i in clauses.args :
            if   i.func == Mul  and isinstance(i.args[0], Number) and  i.args[0] < 0 : 
                negative.append(i) 
    print(negative)
    for i in clauses.args:
        if isinstance(i ,Number):
            constant = int(i)
    print(constant)
    
    if len(negative) > 0 :
        for i in negative :
            if - i.args[0] < constant :  # apply maxsum rule before this so that any higher coeficient goes away.
                var = i / i.args[0]
                expression[var] =0
                print(expression)
            if  - i.args[0] > constant and max_sum(clauses) == i.args[0]:
                for p in  clauses.args and  isinstance(p,Symbol):
                    expression[p] =1
            
    return expression


          

expression_40033={}
renew_clauses2344=[]
'''for clauses in renew_clauses15 :
    rule_41(clauses,expression_3599)'''

for clauses in localrulecheck :
    rulecheck_1(clauses,expression_40033)
for clauses in localrulecheck :
    renew_clauses2344.append(clauses.subs(expression_40033).expand())

renew_clauses2344

In [ ]:
expression_4035 = {}
renew_clauses2356=[]
for clauses in renew_clauses2344 :
    rule_11(clauses,expression_4035)
for clauses in  renew_clauses2344 :
    renew_clauses2356.append(clauses.subs(expression_4035))

renew_clauses2356

In [ ]:
expression_4036 = {}
renew_clauses2357=[]
for clauses in renew_clauses2356 :
    rule_61(clauses,expression_4036)
for clauses in  renew_clauses2356 :
    renew_clauses2357.append(clauses.subs(expression_4036))

renew_clauses2357

In [ ]:
expression_4036[Symbol('p3') * Symbol('q3')] = 0
expression_4036

In [ ]:
#expression_4037 = {}
renew_clauses2358=[]
#for clauses in renew_clauses2358 :
#    rule_71(clauses,expression_4037)
for clauses in  renew_clauses2357 :
    renew_clauses2358.append(clauses.subs(expression_4036))
renew_clauses2358

In [ ]:
expression_4037 = {}
renew_clauses2359=[]
for clauses in renew_clauses2358 :
    rule_11(clauses,expression_4037)
for clauses in  renew_clauses2358 :
    renew_clauses2359.append(clauses.subs(expression_4037))

renew_clauses2359

In [ ]:
expression_4039 = {}
renew_clauses2360=[]
for clauses in renew_clauses2359 :
    rule_71(clauses,expression_4039)
for clauses in  renew_clauses2359 :
    renew_clauses2360.append(clauses.subs(expression_4039))
print(expression_4039)
print(renew_clauses2360)



In [ ]:
renew_clauses17=[]
for clauses in renew_clauses16 :
    rule_21(clauses,expression_2)

for clauses in renew_clauses16 :
    renew_clauses17.append(clauses.subs(expression_2).expand())
renew_clauses17

In [ ]:
renew_clauses18=[]
for clauses in renew_clauses17 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses17 :
    renew_clauses18.append(clauses.subs(expression_2).expand())
renew_clauses18

In [ ]:
renew_clauses19=[]
for clauses in renew_clauses18 :
    rule_41(clauses,expression_2)

for clauses in renew_clauses18 :
    renew_clauses19.append(clauses.subs(expression_2).expand())
renew_clauses19

In [ ]:
renew_clauses20=[]
for clauses in renew_clauses19 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses19 :
    renew_clauses20.append(clauses.subs(expression_2).expand())
renew_clauses20

In [ ]:
renew_clauses21=[]
for clauses in renew_clauses20 :
    rule_41(clauses,expression_2)

for clauses in renew_clauses20 :
    renew_clauses21.append(clauses.subs(expression_2).expand())
renew_clauses21

In [ ]:
renew_clauses22=[]
for clauses in renew_clauses21 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses21 :
    renew_clauses22.append(clauses.subs(expression_2).expand())
renew_clauses22

In [ ]:
renew_clauses23=[]
for clauses in renew_clauses22 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses22 :
    renew_clauses23.append(clauses.subs(expression_2).expand())
renew_clauses23

In [ ]:
renew_clauses24=[]
for clauses in renew_clauses23 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses23:
    renew_clauses24.append(clauses.subs(expression_2).expand())
renew_clauses24

In [ ]:
renew_clauses25=[]
for clauses in renew_clauses24 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses24:
    renew_clauses25.append(clauses.subs(expression_2).expand())
renew_clauses25

In [ ]:
renew_clauses26=[]
for clauses in renew_clauses25 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses25:
    renew_clauses26.append(clauses.subs(expression_2).expand())
renew_clauses26

In [ ]:
renew_clauses27=[]
for clauses in renew_clauses26 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses26:
    renew_clauses27.append(clauses.subs(expression_2).expand())
renew_clauses27

In [ ]:
for key in expression_2:
    if expression_2[key]!=0  :
       renew_clauses27.append(key - expression_2[key])
renew_clauses27

In [ ]:
m,p,q,z=create_dictionary_robust(56153)
p_56153 = create_clause1(m,p,q,z)
p_56153

In [ ]:
expression_56153= {}
renew_clause56153=[]
for clauses in p_56153:
    rule_11(clauses,expression_56153)
for clauses in p_56153:
    renew_clause56153.append(clauses.subs(expression_56153).expand())
renew_clause56153 


In [ ]:
expression_1={}
expression_1[Symbol('q1')] = Symbol('p1')
renew_clause_1=[]
for clause in renew_clause56153:
    renew_clause_1.append(clause.subs(expression_1))
renew_clause_1


In [ ]:
expression_2={}
renew_clause_2=[]
expression_2[Symbol('p2')  + Symbol('q2')] = 1
expression_2[Symbol('p2') * Symbol('q2')] = 0
for clause in renew_clause_1:
    renew_clause_2.append(clause.subs(expression_2))
renew_clause_2


In [ ]:
renew_clause_3=[]
for clause in renew_clause_2:
    renew_clause_3.append(rule_xpow2(clause))
renew_clause_3
    

In [ ]:
expression_3={}
renew_clause_4=[]
expression_3[Symbol('p5')  + Symbol('q5')]=1
expression_3[Symbol('p5')*Symbol('q5')]=0
expression_3[Symbol('q2')]=Symbol('p2')
for clause in renew_clause_1:
    renew_clause_4.append(clause.subs(expression_3))
renew_clause_4


In [ ]:
renew_clause_5=[]
for clause in renew_clause_4:
    renew_clause_5.append(rule_xpow2(clause))
renew_clause_5